In [54]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas
import numpy as np
import requests

import warnings
#warnings.filterwarnings('ignore');
pd.set_option('display.max_rows', 500)

import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import webbrowser

import osmnx as ox
import folium

#### importar datasets finales

In [63]:
location_min_dataset = "../datasets/final_df_min_distance_optimizated.csv"
def import_min_dataset(location_min_dataset):
    interaction_min_dataset = pd.read_csv(location_min_dataset, sep=',', index_col=False)
    return interaction_min_dataset

In [64]:
interaction_min_dataset = import_min_dataset(location_min_dataset)
interaction_min_dataset

,Place of interest,Type of place,Place address,Place distrit,Place neighborhood,Place description,Place transport,Place latitude,Place longitude,mercator place,...,Station location,Station latitude,Station longitude,Station base availability,Station Bikes Availability,mercator station,Distance Between BiciMAD station and Place of interest,place_maps,bici_maps,g_maps
0,Jardines Gregorio Ordóñez,Principales parques y jardines municipales,PRINCIPE DE VERGARA,SALAMANCA,LISTA,Jardines en memoria de Gregorio Ordóñez. Este ...,"Metro: Núñez de Balboa (líneas 5 y 9) Bus: 1, ...",40.433462,-3.678595,POINT (4501032.386132487 -409780.9987743839),...,Calle Diego de León nº 52,40.434597,-3.678492,disponible,13,POINT (4501158.781867075 -409769.4558894613),1.269217e+02,Jardines+Gregorio+Ordóñez,Calle+Diego+de+León+nº+52,https://www.google.com/maps/dir/Jardines+Grego...
1,Jardines San Francisco el Grande,Principales parques y jardines municipales,GRAN VIA DE SAN FRANCISCO,CENTRO,PALACIO,Este jardín se encuentra en el solar de 4.200 ...,"Bus: 3, 60 y 148. Servicio Bicimad, para cons...",40.407143,-3.712167,POINT (4498102.554322098 -413525.9071853098),...,Glorieta de la Puerta de Toledo nº 1,40.407036,-3.711051,disponible,8,POINT (4498090.649721576 -413401.4898074972),1.249856e+02,Jardines+San+Francisco+el+Grande,Glorieta+de+la+Puerta+de+Toledo+nº+1,https://www.google.com/maps/dir/Jardines+San+F...
2,Jardines de Andrés Saborit,Principales parques y jardines municipales,RAIMUNDO FERNANDEZ VILLAVERDE,CHAMBERI,RIOS ROSAS,Jardines situados en la calle Raimundo Fernánd...,"Metro: Nuevos Ministerios (líneas 6, 8 y 10) B...",40.446133,-3.698970,POINT (4502442.959942039 -412053.7810488821),...,Calle Raimundo Fernández Villaverde nº 33,40.447125,-3.700167,disponible,9,POINT (4502553.359051885 -412187.3054398999),1.732534e+02,Jardines+de+Andrés+Saborit,Calle+Raimundo+Fernández+Villaverde+nº+33,https://www.google.com/maps/dir/Jardines+de+An...
3,Jardines de El Buen Retiro,Principales parques y jardines municipales,INDEPENDENCIA,RETIRO,LOS JERONIMOS,Jardín Histórico Artístico creado en el siglo ...,"Metro: Retiro (línea 2), Príncipe de Vergara (...",40.419641,-3.687934,POINT (4499493.840154874 -410822.6776250893),...,Plaza de la Independencia nº 6,40.419752,-3.688398,disponible,19,POINT (4499506.210630401 -410874.469478817),5.324871e+01,Jardines+de+El+Buen+Retiro,Plaza+de+la+Independencia+nº+6,https://www.google.com/maps/dir/Jardines+de+El...
4,Jardines de Gloria Fuertes,Principales parques y jardines municipales,MADRE DE DIOS,CHAMARTIN,NUEVA ESPAÑA,Jardín dedicado a la poetisa y escritora de cu...,"Metro: línea 9 (Colombia) Bus: 16,29,150 Serv...",40.461518,-3.675118,POINT (4504155.540151379 -409393.078164072),...,Avenida Pío XII nº 3,40.460410,-3.677120,disponible,11,POINT (4504032.238487074 -409616.4103570179),2.551089e+02,Jardines+de+Gloria+Fuertes,Avenida+Pío+XII+nº+3,https://www.google.com/maps/dir/Jardines+de+Gl...
5,Jardines de José Luis Jiménez Manso,Principales parques y jardines municipales,CAMINO DE LOS VINATEROS,MORATALAZ,MEDIA LEGUA,Parque emblemático de la zona al ser uno de lo...,"Metro: Estrella (línea 9) Bus: 20, 30, 32 Ser...",40.410945,-3.660502,POINT (4498525.813556252 -407762.6743641385),...,Calle Camino de los Vinateros nº 28,40.410530,-3.657790,disponible,3,POINT (4498479.622286306 -407460.1888912669),3.059920e+02,Jardines+de+José+Luis+Jiménez+Manso,Calle+Camino+de+los+Vinateros+nº+28,https://www.google.com/maps/dir/Jardines+de+Jo...
6,Jardines de Perón,Principales parques y jardines municipales,GENERAL PERON,TETUAN,CUATRO CAMINOS,Jardines situados en la avenida General Perón ...,"Metro: Santiago Bernabéu (línea 10) Bus: 3, 5,...",40.452576,-3.695782,POINT (4503160.152133536 -411698.1348104622),...,Calle Orense nº 36,40.454846,-3.694622,disponible,11,POINT (4503412.812312503 -411568.7402199105),2.838664e+02,Jardines+de+Perón,Calle+Orense+nº+36,https://www.google.com/maps/dir/Jardines+de+Pe...
7,Jardines de Sabatini,Principales parqu

In [4]:
def bicimad_station(input_parque,interaction_min_dataset):
    bicimad_station = "la estación de bicis más cercana es: " +str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "BiciMAD station"].tolist()[0])
    return bicimad_station

In [5]:
def bicimad_adress(input_parque,interaction_min_dataset):
    bicimad_adress = "su dirección es: " + str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "Station location"].tolist()[0])
    return bicimad_adress

In [6]:
def bicimad_bikes(input_parque,interaction_min_dataset):
    bicimad_bikes = "hay " + str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "Station Bikes Availability"].tolist()[0]) + " bicis disponibles"
    return bicimad_bikes

In [7]:
def bicimad_dejar_bici(input_parque,interaction_min_dataset):
    bicimad_dejar_bici =str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "Station base availability"].tolist()[0]) + " base para dejar la bici"
    return bicimad_dejar_bici

In [8]:
def place_description(input_parque,interaction_min_dataset):
    place_description = str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "Place description"].tolist()[0])
    return place_description

In [9]:
def place_barrio(input_parque,interaction_min_dataset):
    place_barrio = "su barrio es: " + str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "Place neighborhood"].tolist()[0])
    return place_barrio

In [10]:
def place_transport(input_parque,interaction_min_dataset):
    place_transport = "contiene los siguientes transportes: " + str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "Place transport"].tolist()[0])
    if str(place_transport) == "contiene los siguientes transportes: nan":
        place_transport = "lo siento, no hay transporte público cercano, anda un poco que te vendrá bien"
    elif "Servicio Bicimad" in str(place_transport):
        place_transport = place_transport.split("Servicio Bicimad")[0]
    return place_transport

In [11]:
def bicimad_station_meters(input_parque,interaction_min_dataset):
    bicimad_station = "la estación de bicis más cercana esta a : " + str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "Distance Between BiciMAD station and Place of interest"].tolist()[0]) + " metros"
    return bicimad_station

In [17]:
def wikiparque_email_sender(receiver_email,filename):

    subject = "WikiParque"
    body = "Te adjuntamos en formato CSV la informacion más relevante del conjunto de Paruqes de la Comunidad de Madrid"
    sender_email = "alvarosaezsanchez@gmail.com"
    receiver_email = receiver_email
    password = "jswfzrfqxpxibfzi"
    
    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message["Bcc"] = receiver_email  # Recommended for mass emails
    
    # Add body to email
    message.attach(MIMEText(body, "plain"))
    
    filename = filename
    
    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    
    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)
    
    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )     
    
    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)

In [41]:
def open_wikipedia(input_parque,interaction_min_dataset):
    final_open_maps_url = str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "g_maps"].tolist()[0])
    return final_open_maps_url

In [ ]:
def restaurantes_google_maps(input_parque,interaction_min_dataset):
    final_restaurantes_url = str(interaction_min_dataset.loc[(interaction_min_dataset[("Place of interest")]==input_parque), "g_maps"].tolist()[0])
    return final_open_maps_url

In [60]:
#en desuso
def speak_wikiparque(text_speak, location_speak):
    input_speak = gtts.gTTS(text_speak, lang="es")
    input_speak.save(location_speak)
    speak = playsound(location_speak)
    return speak

In [62]:
interaction_min_dataset["CONTENT-URL"]

KeyError: 'CONTENT-URL'